In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score, KFold
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,precision_recall_curve, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, VotingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from catboost import CatBoostClassifier

In [ ]:
train = pd.read_csv('../input/expresso-churn-prediction-challenge/Train.csv')
test = pd.read_csv('../input/expresso-churn-prediction-challenge/Test.csv')
submission = pd.read_csv('../input/expresso-churn-prediction-challenge/SampleSubmission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.shape, test.shape

In [ ]:
train.info()

In [ ]:
train.nunique()

In [ ]:
train.columns

In [ ]:
(train.isna().sum()/ 2154048) * 100

In [ ]:
test.head()

In [ ]:
train.REGION.dtype

In [ ]:
train.REGION.fillna('missing').value_counts(normalize=True)

In [ ]:
train.REGION.map(train.REGION.fillna('missing').value_counts()/len(train))

In [ ]:
test.info()

In [ ]:
(test.isna().sum()/380127 ) * 100

train = train.fillna(-999)
test = test.fillna(-999)

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
train.columns

In [ ]:
drop_col = ['user_id']

In [ ]:
new_train = train.drop(drop_col, axis=1)

In [ ]:
new_test= test.drop(drop_col, axis=1)

In [ ]:
target = new_train.CHURN

In [ ]:
concated= pd.concat([new_train.drop('CHURN',axis=  1), new_test]).reset_index(drop = True)

In [ ]:
concated

In [ ]:
for column in concated.select_dtypes('O').columns:
    concated[column] = concated[column].fillna('missing')
    concated[column]  = concated[column].map(concated[column].value_counts(normalize = True))

In [ ]:
concated

concated= pd.get_dummies(concated)

In [ ]:
ntrain = train.shape[0]

In [ ]:
new_train_dummied= concated.iloc[:ntrain]
new_test_dummied= concated.iloc[ntrain:]

In [ ]:
new_test_dummied.head()

In [ ]:
cols = new_train_dummied.columns.difference(['CHURN'])
X= new_train_dummied[cols]

#y= new_train_dummied['CHURN']

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,target, test_size= 0.3, random_state= 42)

In [ ]:
X.columns

X

In [ ]:
X.shape 

In [ ]:

%%time
scaler = StandardScaler()

X_train[X.columns] = scaler.fit_transform(X_train[X.columns])
X_test[X.columns] = scaler.fit_transform(X_test[X.columns])

new_test_dummied[X.columns] = scaler.transform(new_test_dummied[X.columns])

# Logistic Regression

In [ ]:
lr = LogisticRegression()
lr

In [ ]:
%%time
lr.fit(X_train,y_train)

In [ ]:
%%time
lr_y_pred = lr.predict_proba(X_test)[:,1]

In [ ]:
print("roc_auc_score")
roc_auc_score(y_test, lr_y_pred)

In [ ]:
test_pred_lr= lr.predict(new_test_dummied[X.columns])

# Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42,n_jobs=-1)

In [ ]:
%%time
rf.fit(X_train,y_train)

In [ ]:
%%time
rf_y_pred = rf.predict(X_test)

In [ ]:
print("roc_auc_score")
roc_auc_score(y_test, rf_y_pred)

In [ ]:
test_pred_rf= rf.predict(new_test_dummied[X.columns])

In [ ]:
test_pred_rf

# LGBM

In [ ]:
lgb = LGBMClassifier( random_state=42,n_jobs=-1)

In [ ]:
lgb.fit(X_train, y_train)

In [ ]:
lgbb_pred = lgb.predict_proba(X_test)[:,1]

In [ ]:
print("roc_auc_score")
roc_auc_score(y_test, lgbb_pred)

In [ ]:
test_pred_lgbb= lgb.predict_proba(new_test_dummied[X.columns])[:,1]

> XGBOOST

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(random_state=42)

In [ ]:
xgb.fit(X_train, y_train)

In [ ]:
xgb_pred = xgb.predict_proba(X_test)[:,1]

In [ ]:
print("roc_auc_score")
roc_auc_score(y_test, xgb_pred)

In [ ]:
test_pred_xgb= xgb.predict_proba(new_test_dummied[X.columns])[:,1]

In [ ]:
X.shape,new_test_dummied.shape

In [ ]:
new_test_dummied.columns

In [ ]:
X.columns

new_test_dummied = new_test_dummied.drop(['CHURN'], axis=1)

In [ ]:
new_test_dummied

In [ ]:
def cross_validation(X,y,test,model,folds = 5,random_state =  42):
    np.random.seed(42)
    kf = KFold(n_splits=folds,shuffle=True, random_state=random_state) 
    y_copy = y.copy()
    total_test_pred = []
    for train_index, test_index in kf.split(X):
        #print("Train:", train_index, "Validation:",test_index)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index] 
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
        
        model.fit(X_train,y_train)
        
        val_pred  = model.predict_proba(X_test)[:,1]
        y_copy.iloc[test_index] = val_pred
        print('roc_auc_score',roc_auc_score(y_test,val_pred))
        test_pred = model.predict_proba(test)[:,1]
        total_test_pred.append(test_pred)
    total_test_pred = np.mean(total_test_pred,axis = 0)
    print('roc_auc_score', roc_auc_score(y,y_copy))
    return y_copy,total_test_pred

In [ ]:
%%time
total_val_lgb, total_test_lgb = cross_validation(X,target,new_test_dummied, LGBMClassifier( random_state=42,n_jobs=-1),2)

In [ ]:
345678

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
cbc= CatBoostClassifier(iterations= 2000, random_state= 42)

In [ ]:
cbc.fit(X,target, eval_set= (X,target), use_best_model= True)

In [ ]:
cbc_pred = cbc.predict_proba(new_test_dummied)[:,1]

In [ ]:
lgbm= LGBMClassifier(n_estimators=100,verbose = 10, random_state= 42)

In [ ]:
lgbm.fit(X,target, )#eval_set= (X,target))

In [ ]:
a = lgbm.predict_proba(new_train_dummied)[:,1]

In [ ]:
roc_auc_score(target,a)

In [ ]:
lgbm_pred = lgbm.predict_proba(new_test_dummied)[:,1]

In [ ]:
xgbm= XGBClassifier( random_state= 42)

In [ ]:
xgbm.fit(X,target)

In [ ]:
xgbm_pred = xgbm.predict_proba(new_test_dummied)[:,1]

In [ ]:
roc_auc_score(y,total_val_lgb)

In [ ]:
y_train.head()

for col in X_train.columns:
    print(col)

In [ ]:
%%time
lgb.fit(X_train.values,y_train.values)

X_test

In [ ]:
%%time
lgb_y_pred = lgb.predict_proba(X_test.values)[:,1]

In [ ]:
print("roc_auc_score")
roc_auc_score(y_test, lgb_y_pred)

In [ ]:
print("roc_auc_score")
roc_auc_score(y_test, lgb_y_pred)

In [ ]:
test_pred_lgb= lgb.predict_proba(new_test_dummied[X.columns])[:,1]

In [ ]:
test_pred_lgb

In [ ]:
test_pred[:5]

In [ ]:
test.shape

In [ ]:
new_test_dummied.shape

In [ ]:
submission.shape

In [ ]:
submission.head()

In [ ]:
submission["CHURN"] = test_pred_xgb

In [ ]:
submission.to_csv('xgb_train_test_split.csv', index=False)